<div align='center'>

# Practica 3

<img src='https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExdjltdWlscHI5aGY2Ymk4Y2drYWJ0Z21mbzNvbzhoZnFwc3psenl2byZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/QMHoU66sBXqqLqYvGO/giphy.gif'>

</div>

## Ejercicio 1
El dataset Jacobi tiene los coeficientes de un sistema de 15 ecuaciones de 15
incógnitas. Las ecuaciones en el archivo ya están "despejadas" como lo requiere el
método de Jacobi. Cada línea del archivo posee:
<var_N, term_ind, coef_var1, coef_var2, … , coef_ var15>
Por simplicidad, para cada variable N su correspondiente coeficiente es cero.
Implemente en MapReduce el cálculo del método de Jacobi para la solución del
sistema de ecuaciones dado.

---

### Respuesta

* El método de **Jacobi** es **iterativo**: cada valor de `xᵢ` se calcula usando los de la iteración anterior.

* El **dataset** tiene una fila por ecuación:

  ```
  <var_N, término_ind, coef_var1, …, coef_var15>
  ```

  (el coeficiente de la variable `N` es 0 porque ya está despejada).

* En **MapReduce**:

  * **Map**: para cada fila, con el vector `x` anterior calcula el nuevo valor de la variable:

    ```
    xN = (b - Σ coef_varK * xK) / aNN
    ```

    y emite `(var_N, xN)`.
  * **Reduce**: identidad, escribe los nuevos valores `(var_N, xN)`.

* El **driver** controla el bucle:

  * Inicializa `x` (ej: todo en 0).
  * Ejecuta el Job de Jacobi pasando `x` como parámetro (`setParams`).
  * Lee la salida y arma el nuevo vector `x`.
  * Repite hasta que la diferencia con el `x` anterior sea menor a un umbral.

* El **DAG** es una secuencia lineal de Jobs:

  ```
  Input Jacobi → Iteración 1 → Iteración 2 → ... → Iteración k → Solución final
  ```

```

Dataset Jacobi
      │
      ▼
 Job 1 (Map: calcula nuevo xᵢ con vector anterior; Reduce: identidad)
      │
      ▼
 Iteración 1 → Iteración 2 → ... → Iteración k
      │
      ▼
  Vector solución final
```




In [6]:
import sys
sys.path.append("..")
from MRE import Job

inputDir = "./practica_3/Jacobi/input/"
outputDir = "./practica_3/Jacobi/out_iter1/"

# ==================== MAP ====================
def fmap_jacobi(var, value, context):
    cols = value.strip().split('\t')
    b = float(cols[1])            # término independiente
    coefs = [float(c) for c in cols[2:]]  # coeficientes
    x_prev = context["x_prev"]    # vector de la iteración anterior

    # índice de la variable actual (var1 → 0, var2 → 1, …)
    idx = int(var.replace("var", "")) - 1

    # Jacobi: xᵢ^(k+1) = bᵢ - Σⱼ aᵢⱼ * xⱼ^(k)
    total = b
    for j, aij in enumerate(coefs):
        total -= aij * x_prev[j]

    context.write(var, total)

def fred_jacobi(var, values, context):
    for v in values:
        context.write(var, v)

# ejemplo: vector inicial todo en 0
x0 = [0.0] * 15

params = {"x_prev": x0}
job = Job(inputDir, outputDir, fmap_jacobi, fred_jacobi)
job.setParams(params)
job.waitForCompletion()

[MRE] INICIANDO ETAPA DE MAPEO...
[Map] var1 -> 0.0
[Map] var11 -> 0.38
[Map] var2 -> 0.05
[Map] var12 -> 0.22
[Map] var3 -> 0.23
[Map] var13 -> -0.36
[Map] var4 -> -0.24
[Map] var14 -> 0.05
[Map] var5 -> -0.26
[Map] var15 -> 0.22
... [más resultados de map omitidos]
[MRE] INICIANDO ETAPA DE REDUCCIÓN...
[Reduce] Clave recibida: var1 -> [0.0]
[Reduce] Clave recibida: var10 -> [-0.06]
[Reduce] Clave recibida: var11 -> [0.38]
[Reduce] Clave recibida: var12 -> [0.22]
[Reduce] Clave recibida: var13 -> [-0.36]
[Reduce] Clave recibida: var14 -> [0.05]
[Reduce] Clave recibida: var15 -> [0.22]
[Reduce] Clave recibida: var2 -> [0.05]
[Reduce] Clave recibida: var3 -> [0.23]
[Reduce] Clave recibida: var4 -> [-0.24]
... [más reduce claves omitidas]
[MRE] FINALIZANDO Y ESCRIBIENDO RESULTADOS EN DISCO...

[MRE] RESULTADOS FINALES DEL JOB:
var1	0.0
var10	-0.06
var11	0.38
var12	0.22
var13	-0.36
var14	0.05
var15	0.22
var2	0.05
var3	0.23
var4	-0.24
... [más resultados omitidos]


True

---

### Ejercicio 2

Resuelva el problema del método de Jacobi con el dataset **jacobi2**.  
Este dataset tiene los datos almacenados de la siguiente forma:
- <incognita_i, coef_i, valor>


Donde para cada incógnita, los términos de la ecuación correspondiente aparecen en distintas tuplas, inclusive el término independiente.  
El valor de `coef_i` es, o bien el nombre de la incógnita afectada por el coeficiente `valor`, o bien el string **"TI"**, haciendo referencia a que valor es el término independiente de dicha ecuación.  

Ejemplo:

| incognita_i | coef_i | valor |
|-------------|--------|-------|
| X           | TI     | 1     |
| X           | Y      | 3     |
| X           | Z      | 0.5   |
| Y           | TI     | -4    |
| Y           | X      | 1/10  |
| Y           | Z      | 1/10  |
| Z           | TI     | 1     |
| Z           | X      | 1/2   |
| Z           | Y      | 1/2   |

**Nota:** Continúe enviando los valores de las incógnitas por parámetros a los *mappers* y *reducers*, según corresponda.


In [5]:
import sys
sys.path.append("..")
from MRE import Job

inputDir = "./practica_3/jacobi2/input/"
outputDir = "./practica_3/jacobi2/out_iter1/"

def fmap_jacobi2(incog, value, context):
    # incog = primera columna (varX)
    parts = value.strip().split('\t')
    coef, val = parts
    val = float(val)
    x_prev = context["x_prev"]

    if coef == "TI":
        # término independiente b_i
        context.write(incog, ("TI", val))
    else:
        # contribución coef * x_prev[coef]
        contrib = val * x_prev.get(coef, 0.0)
        context.write(incog, ("SUM", contrib))

def fred_jacobi2(incog, values, context):
    b = 0.0
    suma = 0.0
    for tag, v in values:
        if tag == "TI":
            b = v
        else:
            suma += v
    nuevo = b - suma
    context.write(incog, nuevo)

# ejemplo: valores iniciales todos en 0
x0 = {"X": 0.0, "Y": 0.0, "Z": 0.0}

params = {"x_prev": x0}
job = Job(inputDir, outputDir, fmap_jacobi2, fred_jacobi2)
job.setParams(params)
job.waitForCompletion()


[MRE] INICIANDO ETAPA DE MAPEO...
[Map] var13 -> ('TI', -45.0)
[Map] var15 -> ('SUM', -0.0)
[Map] var14 -> ('SUM', -0.0)
[Map] var5 -> ('SUM', -0.0)
[Map] var15 -> ('SUM', -0.0)
[Map] var3 -> ('SUM', -0.0)
[Map] var14 -> ('SUM', -0.0)
[Map] var14 -> ('SUM', -0.0)
[Map] var9 -> ('SUM', -0.0)
[Map] var10 -> ('SUM', -0.0)
... [más resultados de map omitidos]
[MRE] INICIANDO ETAPA DE REDUCCIÓN...
[Reduce] Clave recibida: var1 -> [('SUM', 0.0), ('SUM', 0.0), ('SUM', -0.0), ('TI', 1.0), ('SUM', -0.0), ('SUM', -0.0), ('SUM', -0.0), ('SUM', -0.0), ('SUM', 0.0), ('SUM', -0.0), '...']
[Reduce] Clave recibida: var10 -> [('SUM', -0.0), ('SUM', 0.0), ('SUM', -0.0), ('SUM', -0.0), ('SUM', 0.0), ('SUM', -0.0), ('SUM', -0.0), ('SUM', 0.0), ('SUM', -0.0), ('SUM', -0.0), '...']
[Reduce] Clave recibida: var11 -> [('SUM', 0.0), ('SUM', -0.0), ('SUM', -0.0), ('SUM', 0.0), ('SUM', -0.0), ('SUM', -0.0), ('SUM', 0.0), ('SUM', -0.0), ('SUM', 0.0), ('SUM', -0.0), '...']
[Reduce] Clave recibida: var12 -> [('TI',

True

### Ejercicio 3)

Cómo plantearía una solución MapReduce al siguiente algoritmo secuencial:

**i. entrada**

datos: array [1..N] of <int1, int2, ..., intM>


**ii. algoritmo**

```python
error = 0.001; dif = 1; K=5; M=5
prom = [ 0, 0, 0, 0, 0 ]; N=len(lines)
for t in lines:
    v = t.split("\t")
    for m in range(M):
        prom[m] = prom[m] + float(v[m])
C = []

for m in range(K):
    e=[]
    for m in range(M):
        e.append( prom[m] / N + random.random())
    C.append(e)

while dif > error:
    S=[]
    for m in range(K):
        S.append([ [0,0,0,0,0], 0 ])
    for t in lines:
        v = t.split("\t")
        min=9999999
        for q in range(K):
            a=0
            for m in range(M):
                a=a + (C[q][m] - float(v[m]))**2
            if(a < min):
                min = a
            Q = q

    for m in range(M):
        S[Q][0][m]= S[Q][0][m] + float(v[m])
    S[Q][1] = S[Q][1] + 1

    for q in range(K):
        if S[q][1] > 0:
            for m in range(M):
                S[q][0][m]= S[q][0][m] / S[q][1]
    
    dif=0
    for q in range(K):
        for m in range(M):
        dif=dif + (S[q][0][m] - C[q][m])**2
        if(S[q][1] > 0):
            C[q][m] = S[q][0][m]
```

Idea general

El algoritmo es una versión de **K-Means**.
Cada iteración asigna cada punto al centroide más cercano y luego recalcula los centroides.
En MapReduce: **cada iteración es un job completo**.

---

#### Mapper

Asigna un punto al centroide más cercano.

```python
def fmap_kmeans(key, value, context):
    coords = list(map(float, value.strip().split('\t')))
    centroids = context["centroids"]

    best_q, min_dist = None, float("inf")
    for q, c in enumerate(centroids):
        dist = sum((c[m] - coords[m])**2 for m in range(len(coords)))
        if dist < min_dist:
            min_dist = dist
            best_q = q

    context.write(best_q, (coords, 1))
```

---

#### Reducer

Recalcula los centroides promediando los puntos asignados.

```python
def fred_kmeans(cluster_id, values, context):
    M = len(values[0][0])
    sums = [0.0] * M
    count = 0
    for vec, c in values:
        for m in range(M):
            sums[m] += vec[m]
        count += c
    if count > 0:
        new_centroid = [s / count for s in sums]
        context.write(cluster_id, new_centroid)
```

---

#### Driver

1. Inicializa centroides.
2. Ejecuta Job (map→reduce) pasando centroides con `setParams`.
3. Lee los centroides nuevos.
4. Calcula la diferencia con los anteriores (`dif`).
5. Repite hasta que `dif < error`.

---

#### DAG

```
Datos → Iteración 1 (Job) → Centroides nuevos
       → Iteración 2 (Job) → ...
       → Iteración N → Centroides finales
```

